# Baseline Gender Biased Token Classifiers

* Supervised learning
    * Train, Validate, and (Blind) Test Data: under directory `../data/token_clf_data/model_input/`
* Multilabel classification
    * 3 categories of labels:
        1. Person Name: Unknown, Feminine, Masculine (Non-binary not applied during annotation)
        2. Linguistic: Generalization, Gendered Pronoun, Gendered Role
        3. Contextual: Occupation, Omission, Stereotype (Empowering only applied by one annotator and too few times for training)
* Word Embeddings: GloVe 

***

### Table of Contents

[0.](#0) Preprocessing

[1.](#CC) Classifier Chain Models

***

Load necessary libraries:

In [1]:
# For custom functions and variables
import utils, config

# For data analysis
import pandas as pd
import numpy as np
import os, re

# For creating directories
from pathlib import Path

# For classification
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.adapt import MLTSVM
# Base estimators
from sklearn.ensemble import RandomForestClassifier #, ExtraTreesClassifier
# from sklearn.linear_model import LogisticRegression, RidgeClassifier, PassiveAggressiveClassifier

<a id="0"></a>
## 0. Preprocessing

Load the train and validation (dev) data:

In [2]:
df_train = pd.read_csv(config.tokc_path+"model_input/token_train.csv", index_col=0)
df_dev = pd.read_csv(config.tokc_path+"model_input/token_validate.csv", index_col=0)
print(df_train.shape, df_dev.shape)
df_train.head()

(467564, 10) (157740, 10)


,description_id,sentence_id,ann_id,token_id,token,token_offsets,pos,tag,field,subset
3,1,1,99999,3,Title,"(17, 22)",NN,O,Title,train
4,1,1,99999,4,:,"(22, 23)",:,O,Title,train
5,1,1,99999,5,Papers,"(24, 30)",NNS,O,Title,train
6,1,1,99999,6,of,"(31, 33)",IN,O,Title,train
7,1,1,14384,7,The,"(34, 37)",DT,B-Unknown,Title,train


In [3]:
df_train = df_train.drop(columns=["ann_id"])
df_train = df_train.drop_duplicates()
df_dev = df_dev.drop(columns=["ann_id"])
df_dev = df_dev.drop_duplicates()
print(df_train.shape, df_dev.shape)

(463441, 9) (156146, 9)


Remove Non-binary labels as these were mistaken labels identified early on that were meant to be excluded, and because only one token has this label, it prevents the data from being input into the models with cross-validation.

In [4]:
df_train = df_train.loc[df_train.tag != "B-Nonbinary"]
df_train = df_train.loc[df_train.tag != "I-Nonbinary"]

In [5]:
df_train.shape

(463439, 9)

Group the data by token, so there is one row per token rather than one row per token-tag pair:

In [6]:
subdf_train = df_train.drop(columns=["description_id", "field", "subset", "token_offsets"])
subdf_dev = df_dev.drop(columns=["description_id", "field", "subset", "token_offsets"])
df_train_imploded = utils.implodeDataFrame(subdf_train, ["sentence_id", "token_id", "token", "pos"])
df_train_imploded = df_train_imploded.reset_index()
df_dev_imploded = utils.implodeDataFrame(subdf_dev, ["sentence_id", "token_id", "token", "pos"])
df_dev_imploded = df_dev_imploded.reset_index()
df_dev_imploded.head()

,sentence_id,token_id,token,pos,tag
0,5,154,After,IN,[O]
1,5,155,his,PRP$,[B-Gendered-Pronoun]
2,5,156,ordination,NN,[O]
3,5,157,he,PRP,[B-Gendered-Pronoun]
4,5,158,spent,VBD,[O]


#### Word Embeddings

Get GloVe word embeddings (which were trained on English Wikipedia entries) for the vocabulary of the dataset (the unique tokens in the training set):

In [7]:
dimensions = ["50", "100", "200", "300"]
d = dimensions[0]  # compare to custom fastText embeddings with 50 dimensions

In [8]:
glove = utils.getGloveEmbeddings(d)
# print(glove["the"])

In [9]:
vocabulary = list(df_train.token.unique())
vocabulary_lowercased = [token.lower() for token in vocabulary]
vocabulary_lowercased = list(set(vocabulary_lowercased))
print("Vocabulary size:", len(vocabulary))
print("Lowercased vocabulary size:", len(vocabulary_lowercased))

Vocabulary size: 35968
Lowercased vocabulary size: 31335


In [10]:
word_embeddings = utils.getEmbeddingsForTokens(glove, vocabulary)

In [11]:
assert np.array_equal(word_embeddings[0], glove[vocabulary[0].lower()])

In [12]:
embedding_dict = dict(zip(vocabulary, word_embeddings))

In [13]:
embedding_dict_keys = list(embedding_dict.keys())
for token in vocabulary:
    assert token in embedding_dict_keys

Vectorize and binarize the data:

In [14]:
mlb = MultiLabelBinarizer()

In [15]:
target_col = "tag"
feature_cols = ["token_id", "token"]  # try sentence_id?
train_data = df_train_imploded  #.head(int((df_train_imploded.shape[0]/2)))
dev_data = df_dev_imploded      #.head(int((df_dev_imploded.shape[0]/2)))

In [16]:
train_data.head()

,sentence_id,token_id,token,pos,tag
0,1,3,Title,NN,[O]
1,1,4,:,:,[O]
2,1,5,Papers,NNS,[O]
3,1,6,of,IN,[O]
4,1,7,The,DT,"[B-Unknown, B-Masculine, B-Stereotype]"


Extract features:

In [17]:
# Get a vector representation of a token from a GloVe word embedding
def extractEmbedding(token, embedding_dict=glove, dimensions=int(d)):
    if token.isalpha():
        token = token.lower()
    try:
        embedding = embedding_dict[token]
    except KeyError:
        embedding = np.zeros((dimensions,))
    return embedding.reshape(-1,1)

def makeFeatureMatrix(token_data, dimensions=int(d)):    
    feature_list = [extractEmbedding(token) for token_id,token in token_data]
    return np.array(feature_list).reshape(-1,dimensions)

In [18]:
train_tokens = list(zip(train_data[feature_cols[0]], train_data[feature_cols[1]]))
dev_tokens = list(zip(dev_data[feature_cols[0]], dev_data[feature_cols[1]]))

In [19]:
X_train = makeFeatureMatrix(train_tokens)
X_dev = makeFeatureMatrix(dev_tokens)
print(X_train.shape, X_dev.shape)  # (number_of_samples, number_of_features)

(452086, 50) (152455, 50)


Binarize targets:

In [20]:
y_train_labels = train_data[target_col]
y_train = mlb.fit_transform(y_train_labels)
y_dev_labels = dev_data[target_col]
y_dev = mlb.transform(y_dev_labels)
print(y_train.shape, y_dev.shape)  # number_of_samples, number_of_labels

(452086, 19) (152455, 19)


In [21]:
for labels in y_train:
    if sum(labels) > 1:
        print("Multilabelled tokens exist, as expected.")
        break

Multilabelled tokens exist, as expected.


For baseline models, use only the tokens' embeddings as features.

### Twin Support Vector Machines (MLTSVM)

In [22]:
# clf = MLTSVM(c_k = 2**-1)
# clf.fit(X_train, y_train)  # full dataset needs 1.47 TiB of data - try on one category of labels at a time?

In [23]:
# predictions = clf.predict(X_test)

<a id="CC"></a>
## 1. Classifier Chain Models

Classifier Chain Models with Random Forest were best from the experiments with custom fastText embeddings, so use the same model here to compare performance with GloVe embeddings

*Reference: http://scikit.ml/api/skmultilearn.problem_transform.cc.html#skmultilearn.problem_transform.ClassifierChain*

#### Train & Predict

In [24]:
clf = ClassifierChain(
    classifier = RandomForestClassifier(random_state=22),
)
clf.fit(X_train, y_train)

ClassifierChain(classifier=RandomForestClassifier(random_state=22),
                require_dense=[True, True])

In [25]:
predictions = clf.predict(X_dev)

#### Evaluate: All Labels

In [26]:
print("Precision - weighted:", metrics.precision_score(y_dev, predictions, average="weighted", zero_division=0))
print("Precision - macro:", metrics.precision_score(y_dev, predictions, average="macro", zero_division=0))  # macro = mean of all labels' score
print()
print("Recall - weighted:", metrics.recall_score(y_dev, predictions, average="weighted", zero_division=0))
print("Recall - macro:", metrics.recall_score(y_dev, predictions, average="macro", zero_division=0))
print()
print("F1 Score - weighted:", metrics.f1_score(y_dev, predictions, average="weighted", zero_division=0))
print("F1 Score - macro:", metrics.f1_score(y_dev, predictions, average="macro", zero_division=0))
print()
print("Accuracy - normalized:", metrics.accuracy_score(y_dev, predictions, normalize=True))  # fraction of correctly classified samples
print("Accuracy - unnormalized:", metrics.accuracy_score(y_dev, predictions, normalize=False))  # number of correctly classified samples

Precision - weighted: 0.910346142402394
Precision - macro: 0.5009792366083057

Recall - weighted: 0.9254287653862411
Recall - macro: 0.30778935555653825

F1 Score - weighted: 0.9106900294785842
F1 Score - macro: 0.3497917030631622

Accuracy - normalized: 0.935987668492342
Accuracy - unnormalized: 142696


With fastText embeddings of 50 dimensions:
  - Precision - weighted: 0.9139004974205044
  - Precision - macro: 0.511704654080411
  - Recall - weighted: 0.9275485763324068
  - Recall - macro: 0.33277613917269966
  - F1 Score - weighted: 0.9144025420987218
  - F1 Score - macro: 0.3737284450574932
  - Accuracy - normalized: 0.9373388868846545
  - Accuracy - unnormalized: 142902

#### Evaluate: Each Label

In [27]:
pred_df = utils.makePredictionDF(predictions, dev_data, "tag", "predicted_tag", "O", mlb)
pred_df.head()

,sentence_id,token_id,token,pos,predicted_tag
0,5,154,After,IN,O
1,5,155,his,PRP$,B-Gendered-Pronoun
2,5,156,ordination,NN,O
3,5,157,he,PRP,B-Gendered-Pronoun
4,5,158,spent,VBD,O


In [28]:
exp_df = dev_data.explode(["tag"])
exp_df = exp_df.rename(columns={"tag":"expected_tag"})
# exp_df.head()

In [29]:
eval_df = utils.makeEvaluationDataFrame(
    exp_df, 
    pred_df, 
    ["sentence_id", "token_id", "token", "pos", "expected_tag"],   # left on
    ["sentence_id", "token_id", "token", "pos", "predicted_tag"],  # right on
    ["sentence_id", "token_id", "token", "pos", "expected_tag", "predicted_tag", "_merge"],  # final column list
    "expected_tag",
    "predicted_tag", 
    "token_id",  # ID column
    "O"          # No tag value
)
eval_df.head()

,sentence_id,token_id,token,pos,expected_tag,predicted_tag,_merge
0,5,154,After,IN,O,O,true negative
1,5,155,his,PRP$,B-Gendered-Pronoun,B-Gendered-Pronoun,true positive
2,5,156,ordination,NN,O,O,true negative
3,5,157,he,PRP,B-Gendered-Pronoun,B-Gendered-Pronoun,true positive
4,5,158,spent,VBD,O,O,true negative


Save the data:

In [30]:
Path(config.tokc_path+"multilabel_model_output/").mkdir(parents=True, exist_ok=True)
eval_df.to_csv(config.tokc_path+"multilabel_model_output/cc-rf_baseline_GloVe50_predictions.csv")
# eval_df.to_csv(config.tokc_path+"multilabel_model_output/cc-rf_baseline_GloVe{}_predictions.csv".format(d))

##### Strict Agreement

Calculate the total true positives, false positives, true negatives, and false negatives.

In [31]:
agmt_stats = utils.getAgreementStatsForAllTags(eval_df, "_merge", "token_id", "tag(s)", y_dev, predictions)

Calculate precision, recall, and F1 score at the token level for each tag:

In [32]:
label_tags = [ 
    'B-Unknown', 'I-Unknown', 'B-Feminine', 'I-Feminine', 'B-Masculine',  'I-Masculine',
    'B-Gendered-Pronoun', 'I-Gendered-Pronoun','B-Gendered-Role', 'I-Gendered-Role', 
    'B-Generalization', 'I-Generalization', 
    'B-Stereotype', 'I-Stereotype', 'B-Omission', 'I-Omission', 'B-Occupation', 'I-Occupation'
]

In [33]:
for label_tag in label_tags:
    label_agmt_stats = utils.getScoresByTags(eval_df, "_merge", [label_tag])
    agmt_stats = pd.concat([agmt_stats, label_agmt_stats])
agmt_stats

,tag(s),false negative,false positive,true negative,true positive,precision,recall,f1
0,all,11644,9808,140452,4050,0.500979,0.307789,0.349792
0,B-Unknown,1444,310,0,884,0.740369,0.379725,0.501988
0,I-Unknown,2489,471,0,1146,0.708720,0.315268,0.436405
0,B-Feminine,112,69,0,338,0.830467,0.751111,0.788798
0,I-Feminine,615,55,0,120,0.685714,0.163265,0.263736
0,B-Masculine,514,211,0,908,0.811439,0.638537,0.714679
0,I-Masculine,1019,296,0,442,0.598916,0.302533,0.402001
0,B-Gendered-Pronoun,9,178,0,1470,0.891990,0.993915,0.940198
0,I-Gendered-Pronoun,15,0,0,0,0.000000,0.000000,0.000000
0,B-Gendered-Role,181,131,0,788,0.857454,0.813209,0.834746


Save the data:

In [34]:
Path(config.tokc_path+"multilabel_model_performance/").mkdir(parents=True, exist_ok=True)
agmt_stats.to_csv(config.tokc_path+"multilabel_model_performance/cc-rf_baseline_GloVe{}_strict_agmt.csv".format(d))